In [ ]:
from keras.datasets.mnist import load_data
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load the data
(train_digits, train_labels), (test_digits, test_labels) = load_data()

# Set random seed for reproducibility
np.random.seed(123)

# Initialize dictionary to store 5 samples for each digit (0-9)
digit_samples = {digit: [] for digit in range(10)}

# Loop through each digit and randomly select 5 samples from the training set
for digit in range(10):
    # Get all indices of the current digit in train_labels
    digit_indices = np.where(train_labels == digit)[0]

    # Randomly select 5 indices from these
    random_indices = np.random.choice(digit_indices, 5, replace=False)

    # Collect the images of the chosen indices
    digit_samples[digit] = train_digits[random_indices]

# Plot the samples in a 10x5 grid
plt.figure(figsize=(10, 10))

for digit, images in digit_samples.items():
    for i, img in enumerate(images):
        plt.subplot(10, 5, digit * 5 + i + 1)  # Position in 10x5 grid
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        if i == 0:
            plt.ylabel(f"Digit {digit}", fontsize=12)

plt.suptitle("5 Random Samples of Each Digit (0-9)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


In [ ]:
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Define the path to your dataset in Google Drive
# Replace with your actual path inside your Google Drive
data_dir = '/content/drive/MyDrive/MNIST_Dataset'

# Step 2: Prepare the Image Data
# Using ImageDataGenerator to load images and apply preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,           # Normalize pixel values between 0 and 1
    validation_split=0.2         # Use 20% of data for validation
)

# Load training data from the folders
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),        # Resize images to 28x28 pixels
    color_mode="grayscale",      # Convert images to grayscale
    batch_size=32,
    class_mode="categorical",    # Use categorical labels (one-hot encoding)
    subset="training"            # Training subset
)

# Load validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    color_mode="grayscale",
    batch_size=32,
    class_mode="categorical",
    subset="validation"          # Validation subset
)

# Step 3: Build the Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # Output layer with 10 classes
])

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Train the Model
epochs = 10  # Number of epochs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Step 5: Evaluate the Model (optional)
loss, accuracy = model.evaluate(validation_generator)
print("Validation Accuracy:", accuracy)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical

# Load the MNIST dataset
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() # Load test data

# ... (rest of your existing code for X_train processing) ...

# Some variables...
image_height = train_digits.shape[1]
image_width = train_digits.shape[2]
num_channels = 1  # we have grayscale images

# Reshape the images data
train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0], image_height, image_width, num_channels))

# Rescale the image data to values between (0.0,1.0]
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

# One-hot encode the labels
num_classes = 10
train_labels_cat = to_categorical(train_labels, num_classes)
test_labels_cat = to_categorical(test_labels, num_classes)

In [ ]:
# shuffle the training dataset (5 times!)
for _ in range(5):
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

# now set-aside 10% of the train_data/labels as the
# cross-validation sets
val_perc = 0.10
val_count = int(val_perc * len(train_data))

# first pick validation set from train_data/labels
val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

# leave rest in training set
train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]

# NOTE: We will train on train_data2/train_labels_cat2 and
# cross-validate on val_data/val_labels_cat

In [ ]:
# Import matplotlib to create plots
import matplotlib.pyplot as plt

def show_plots(history):
    """
    Displays plots for accuracy and loss during training.

    Args:
        history: The training history object returned by model.fit.
    """

    # Plot training & validation accuracy values
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from google.colab import drive

# Mount Google Drive to save the trained model
drive.mount('/content/drive')

# Load the MNIST dataset
(train_digits, train_labels), (test_digits, test_labels) = keras.datasets.mnist.load_data()

# Preprocess the data
train_digits = train_digits.reshape((train_digits.shape[0], 28, 28, 1)).astype("float32") / 255
test_digits = test_digits.reshape((test_digits.shape[0], 28, 28, 1)).astype("float32") / 255

# Convert labels to one-hot encoded format
train_labels_cat = to_categorical(train_labels, 10)
test_labels_cat = to_categorical(test_labels, 10)

# Build a CNN model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Instantiate and train the model
model = build_model()
model.fit(train_digits, train_labels_cat, epochs=10, batch_size=64, validation_data=(test_digits, test_labels_cat))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_digits, test_labels_cat)
print("Test accuracy:", test_accuracy)

# Save the model to Google Drive
model.save('/content/drive/MyDrive/MNIST_Dataset/my_model.h5')  # Replace with your desired model path and name
print("Model saved successfully!")

In [ ]:
results = model.fit(train_data2, train_labels_cat2,
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

In [ ]:
# display plots...
show_plots(results.history)

In [ ]:
test_loss, test_accuracy = \
  model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

In [ ]:
predictions = model.predict(test_data)
first20_preds = np.argmax(predictions, axis=1)[:25]
first20_true = np.argmax(test_labels_cat,axis=1)[:25]
print(first20_preds)
print(first20_true)

In [ ]:
# how many mismatches?
num_mismatches = (np.argmax(predictions, axis=1) != np.argmax(test_labels_cat, axis=1)).sum()
print("Number of mismatches:", num_mismatches)

In [ ]:
!pip install Pillow

In [ ]:
# Import required libraries
import numpy as np
from PIL import Image
import os
from google.colab import drive
from tensorflow import keras

# Mount Google Drive
drive.mount('/content/drive')

# Load the trained model (change 'your_model_path' to the actual path)
model = keras.models.load_model('/content/drive/MyDrive/MNIST_Dataset/my_model.h5', compile=False)

# Define a function to preprocess the image and make predictions
def preprocess_and_predict(image_path, model, threshold=0.1):
    """Loads an image, preprocesses it, and predicts using the model."""
    try:
        # Load the image
        input_image = Image.open(image_path).convert('L')  # Convert to grayscale
        input_image_resize = input_image.resize((28, 28))  # Resize to 28x28

        # Convert image to array and normalize
        input_image_array = np.array(input_image_resize) / 255.0
        image_reshaped = np.reshape(input_image_array, (1, 28, 28, 1))  # Reshape to (1, 28, 28, 1)

        # Make a prediction
        prediction = model.predict(image_reshaped)
        max_prob = np.max(prediction)
        predicted_label = np.argmax(prediction)

        # Check threshold
        if max_prob >= threshold:
            return predicted_label, max_prob
        else:
            return "Uncertain", max_prob
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None

# Folder path in Google Drive containing images (replace with actual folder path)
image_folder_path = '/content/drive/MyDrive/MNIST_Images/'

# Loop through each image in the folder and predict
for image_filename in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_filename)

    if image_path.endswith(('.png', '.jpg', '.jpeg')):  # Check for valid image files
        predicted_label, confidence = preprocess_and_predict(image_path, model, threshold=0.9)
        print(f"Image: {image_filename} - Predicted Digit: {predicted_label}, Confidence: {confidence}")
